In [5]:
import pandas as pd
import numpy as np
from datasketch import MinHash, MinHashLSH
import pickle

df = pd.read_csv('spotify_dataset/data.csv')
df

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.9950,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563000,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.9940,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901000,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.6040,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.9950,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887000,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.9900,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908000,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,0.1730,"['DripReport', 'Tyga']",0.875,163800,0.4430,1,4KppkflX7I3vJQk7urOJaS,0.000032,1,0.0891,-7.461,1,Skechers (feat. Tyga) - Remix,75,2020-05-15,0.1430,100.012,0.3060,2020
169905,0.0167,"['Leon Bridges', 'Terrace Martin']",0.719,167468,0.3850,0,1ehhGlTvjtHo2e4xJFB0SZ,0.031300,8,0.1110,-10.907,1,Sweeter (feat. Terrace Martin),64,2020-06-08,0.0403,128.000,0.2700,2020
169906,0.5380,"['Kygo', 'Oh Wonder']",0.514,180700,0.5390,0,52eycxprLhK3lPcRLbQiVk,0.002330,7,0.1080,-9.332,1,How Would I Know,70,2020-05-29,0.1050,123.700,0.1530,2020
169907,0.0714,"['Cash Cash', 'Andy Grammer']",0.646,167308,0.7610,0,3wYOGJYD31sLRmBgCvWxa4,0.000000,1,0.2220,-2.557,1,I Found You,70,2020-02-28,0.0385,129.916,0.4720,2020


In [19]:
df = df[['id','acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness','speechiness', 'tempo', 'valence', 'year']]

In [20]:
df2 = df[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness','speechiness', 'tempo', 'valence', 'year']]

In [21]:
def create_minhash(data, num_perm):
    min_hash = []
    m = MinHash(num_perm = num_perm)
    for d in data:
        m.update(str(d).encode('utf8'))
    min_hash.append(m)
    return min_hash

In [22]:
def query_lsh(minhash, lsh_signature):
    results = []
    for idx, h in enumerate(minhash):
        result = lsh_signature.query(h)
        results.append(result)
    return(results)

In [29]:
lsh_signatures = pickle.load(open('lsh_signatures_0.3.sav', 'rb'))

In [24]:
def display_nearest_neighbours(results, df):
    for idx, res in enumerate(results):
        temp_lst = []
        for i in res:
            temp_lst.append(df.iloc[i])
        print(temp_lst)

In [25]:
def get_nearest_neighbours(results, df):
    for idx, res in enumerate(results):
        nn = []
        for i in res:
            nn.append(df.iloc[i])
    return nn

def get_NN_id(results, df):
    return get_nearest_neighbours(results, df)[0]['id']

def get_random_id(df):
    return df.sample().iloc[0]['id']

In [26]:
def full_lsh(data, num_perm, df):
    min_hash = create_minhash(data, num_perm)
    results = query_lsh(min_hash, lsh_signatures)
    nn = get_NN_id(results, df)
    return nn
full_lsh([0.9950,0.2,158648.0000,0.1950,0.20,0.1510,-12.4280,0.0506,118.4690,0.7790,2020], 6, df)

'1wIQtB3UQ1TfjNMZZqO6eh'

In [27]:
d = [0.48767, 0.5635, 175159.0, 0.26065, 4.055e-06, 0.2945, -12.431, 0.28295, 113.08000000000001, 0.1815, 1975.5]
def find_best_match(data):
    id = ''
    min_score = np.Inf
    for index, row in df.iterrows():
        curr_score = 0
        for col in range(1, len(row)-1):
            curr_score += abs(data[col] - row[col])
        if (min_score > curr_score):
            min_score = curr_score
            id = row[0]
            print(id, curr_score)
    return id

find_best_match(d)

6KbQ3uYMLKb5jDxLF7wYDD 336038.257195945
6L63VW0PibdM1HDSBoqnoM 281685.06879594503
6QIONtzbQCbnmWNwn0H1yT 262509.62059594505
6UxGTlqovmrBV4SKAsoz0A 258158.39769594502
74qEIULd05bBGGtWiCglvC 254346.23989594504
75zIt9muIwtL6o30PcqskO 225985.47469594495
40q1O23P19AcDSghky2Ct3 207347.56579594497
2ndQKlOL0BCycoGoNFRFo0 195267.957095945
2mex2o4uA69pMcLjMtyyGb 191987.11405194498
26hYdUWilVb0S2nurNtcj4 189255.99679594496
0Yu1hwkNgqwklv8JJlI3Et 186124.98279594502
3lRVIn6D6EUbvkOgPZAU1H 183788.07280405497
3IcXTeq9O2dpsSXsDj9naH 183312.07280405497
0hr9kRUi2X4MXc72A4VxG4 182429.07280405497


'0hr9kRUi2X4MXc72A4VxG4'

In [28]:

def create_minhash_signatures(df, lsh_threshold, num_perm):
    min_hashes = []
    for i in range(len(df)):
        row = df.iloc[i]
        m = MinHash(num_perm = num_perm)
        for s in row:
            m.update(str(s).encode('utf8'))
        min_hashes.append(m)
        
    lsh = MinHashLSH(lsh_threshold, num_perm) 
    for key, minhash in enumerate(min_hashes):
        lsh.insert(key, minhash)
    return(min_hashes, lsh)

minhashes, lsh_signatures = create_minhash_signatures(df2, lsh_threshold = 0.3, num_perm = 6)

KeyboardInterrupt: 

In [30]:
data = []
for index, row in df.iterrows():
    d = list(row)[1:]
    min_hash = create_minhash(d, 6)
    results = query_lsh(min_hash, lsh_signatures)
    data.append([row[0], results, len(results[0])])

KeyboardInterrupt: 

In [ ]:
df_results = pd.DataFrame(data=data, columns=['id', 'results', 'len'])

In [ ]:
df_results[df_results['len'] == 0]

In [32]:
# pickle.dump(lsh_signatures, open('lsh_signatures_0.2.sav', 'wb'))
score = dict()
count_zero = 0
count_non = 0
def configure_score(id, b):
    row = df[df['id'] == id]
    row = row[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness','speechiness', 'tempo', 'valence', 'year']]
    if (b):
        for col in row.columns:
            if (col not in score.keys()):
                    score[col] = [float(row.iloc[0][col]), 1]
            else:
                avg = score[col][0]
                num = score[col][1]
                new_avg = float("{:.2f}".format(((float(row.iloc[0][col]) + avg * num) / (num+1))))
                # new_avg = float(row.iloc[0][col])
                score[col] = [new_avg, num+1]

def loop(count_zero, count_non):
    spotify_id = get_random_id(df)
    configure_score(spotify_id, True)
    data = []
    for key in score.keys():
        data.append(score[key][0])
    min_hash = create_minhash(data, 6)
    results = query_lsh(min_hash, lsh_signatures)
    if (len(results[0]) == 0):
        count_zero+= 1
        print(count_zero, count_non)
        loop(count_zero, count_non)
    else:
        count_non += 1
        print(count_zero, count_non)
        loop(count_zero, count_non)


loop(0, 0)


0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 113
1 114
1 115
1 116
1 117
1 118
1 119
1 120
1 121
1 122
1 123
1 124
1 125
1 126
1 127
1 128
1 129
1 130
1 131
1 132
1 133
1 134
1 135
1 136
1 137
1 138
1 139
1 140
1 141
1 142
1 143
1 144
1 145
1 146
1 147
1 148
1 149
1 150
1 151
1 152
1 153
1 154
1 155
1 156
1 157
1 158
1 159
1 160
1 161
1 162
1 163
1 164
1 165
1 166
1 167
1 168
1 169
1 170
1 171
1 172
1 173
1 174
1 175
1 176
1 177
1 178
1 179
1 180
1 181
1 182
1 183
1 184

KeyboardInterrupt: 

In [51]:
pickle.dump(lsh_signatures, open('lsh_signatures_0.3.sav', 'wb'))